##### 数据流的伪代码

In [ ]:
#训练流
for data_batch in train_dataloader:
    data_batch=model.preprocessor(data_batch)
    if isinstance(data_batch,dict):
        losses=model(**data_batch,mode='loss')
    elif isinstance(data_batch,(list,tuple)):
        losses=model(*data_batch,mode='loss')
    else:
        raise TypeError()
#验证流
for data_batch in val_dataloader:
    data_batch=model.preprocessor(data_batch)
    if isinstance(data_batch,dict):
        outputs=model(**data_batch,mode='predict')
    elif isinstance(data_batch,(list,tuple)): 
        outputs=model(*data_batch,mode='predict')
    else:
        raise TypeError()
    evaluator.process(data_samples=outputs,data_batch=data_batch)

metrics=evaluator.evaluate(len(val_dataloader.dataset))

1. 继承自BaseModel的模型需要实现forward方法；
2. BaseModel内部实现了train_step、val_step和test_step方法，在三种方法的内部首先使用模型的数据预处理器
    preprocessor将输入数据转换为模型输入的格式，然后调用forward方法,forward方法对数据源使用模型处理，根据mode参数
    返回损失（训练流）、预测结果（验证流和测试流），然后在*_step方法内部处理forward的返回结果。

##### 数据流的伪代码2

In [ ]:
#训练流
for data_batch in train_dataloader:
    losses=model.train_step(data_batch)
#验证流
for data_batch in val_dataloader:
    predicts=model.val_step(data_batch)
    evaluator.process(data_samples=predicts,data_batch=data_batch)
evaluator.evaluate(len(val_dataloader.dataset))

##### 完整的模型内部数据流

In [ ]:
import torch.nn.functional as F
import torchvision
from mmengine.model import BaseModel

class MMResNet50(BaseModel):
    def __init__(self):
        super().__init__()
        self.resnet50=torchvision.models.resnet50()
    def forward(self,imgs):
        pass

    #BaseModel内部已经实现了这三个方法
    def train_step(self,data_batch,optim_wrapper):
        data_batch=self.data_preprocessor(data_batch)
        losses=self.forward(*data_batch,mode='loss')#CIFAR10返回tuple
        parsed_loss,log_vars=self.parse_losses(losses)
        optim_wrapper.update_params(parsed_loss)    
        return log_vars
    
    def val_step(self,data_batch,optim_wrapper):
        data_batch=self.data_preprocessor(data_batch)
        predicts=self.forward(*data_batch,mode='predict')
        return predicts
    
    def test_step(self,data_batch,optim_wrapper):
        data_batch=self.data_preprocessor(data_batch)
        predicts=self.forward(*data_batch,mode='predict')
        return predicts


##### 数据预处理器（DataPreprocessor）
1. 继承nn.Module，用于将数据搬运至指定设备，如GPU；
2. 先搬运数据，再做归一化，因为归一化后是浮点数，空间更大，不利于搬运；
3. 做批图像增强，单张图像的图像增强如翻转，裁剪等在transforms中做合适，而需要用到多张图像的融合时在这里做合适；
4. dataloader的输出和模型输入不匹配时，可在此调整。